In [1]:
import networkx as nx
import ibm_db
import ibm_db_dbi
import pandas as pd
from pyvis.network import Network

In [2]:
#conectando ao BigSql
#COLOCAR CREDENCIAIS
db = ibm_db.connect("DATABASE=BIGSQL;HOSTNAME=bigsql.pro.intra.rs.gov.br;PORT=32051;PROTOCOL=TCPIP;UID=XXXXXXXX;PWD=XXXXXX;", "", "")
conn = ibm_db_dbi.Connection(db) 

In [3]:
query = """SELECT 
                PO.NRO_INT_OCORRENCIA,
                PO.NRO_INT_PESSOA_INTERESSE,
                TP.TXT_DESCRICAO AS TIPO_PARTICIPACAO
            FROM
                GESEG_TARGET.GESEG_VER_OCORRENCIA O
                INNER JOIN GESEG_TARGET.GESEG_VER_PARTICIPANTE_OCORRENCIA PO ON O.NRO_INT_OCORRENCIA = PO.NRO_INT_OCORRENCIA
                INNER JOIN GESEG_TARGET.GESEG_DIM_TIPO_PARTICIPACAO TP ON PO.NRO_INT_TIPO_PARTICIPACAO = TP.NRO_INT_TIPO_PARTICIPACAO

            WHERE
                IND_TENTATIVA = 'Consumado'	
                AND YEAR(DATA_FATO) = 2021
                AND MONTH(DATA_FATO) = 1
                AND DAY(DATA_FATO) = 18;"""

In [4]:
df = pd.read_sql_query(query, conn)
# alterando os nomes das colunas para letras minúsculas
df.rename(str.lower, axis='columns', inplace = True)
df.head()

,nro_int_ocorrencia,nro_int_pessoa_interesse,tipo_participacao
0,38768775,9891686,Autor
1,38563542,14079520,Só comunicante
2,38119484,3314386,Só comunicante
3,38123996,5373829,Só comunicante
4,38108963,6178835,Só comunicante


In [5]:
#informando quais colunas da tabela serão o source, target e demais atributos;
D = nx.from_pandas_edgelist(df, source = 'nro_int_ocorrencia', target = 'nro_int_pessoa_interesse', edge_attr=['tipo_participacao'], create_using=nx.MultiDiGraph())

In [6]:
# verificando se importou os edges corretamente
D.edges(data = True)

OutMultiEdgeDataView([(38768775, 9891686, {'tipo_participacao': 'Autor'}), (38768775, 12292980, {'tipo_participacao': 'Só comunicante'}), (38768775, 7934779, {'tipo_participacao': 'Vítima'}), (38768775, 2279906, {'tipo_participacao': 'Testemunha'}), (38768775, 5904344, {'tipo_participacao': 'Testemunha'}), (38563542, 14079520, {'tipo_participacao': 'Só comunicante'}), (38563542, 14079520, {'tipo_participacao': 'Vítima'}), (38119484, 3314386, {'tipo_participacao': 'Só comunicante'}), (38119484, 3314386, {'tipo_participacao': 'Vítima'}), (38123996, 5373829, {'tipo_participacao': 'Só comunicante'}), (38108963, 6178835, {'tipo_participacao': 'Só comunicante'}), (38108963, 6178835, {'tipo_participacao': 'Vítima'}), (38098944, 1707143, {'tipo_participacao': 'Só comunicante'}), (38098944, 9485593, {'tipo_participacao': 'Vítima'}), (38097662, 8351774, {'tipo_participacao': 'Só comunicante'}), (38126960, 10937243, {'tipo_participacao': 'Só comunicante'}), (38126960, 10937243, {'tipo_participaca

In [7]:
# definindo o atributo 'tipo_no' para todos os nós
for n in D.nodes:
    D.nodes[n].setdefault('tipo_no', 'sem_descricao')

In [8]:
#testando
sorted(D.nodes(data = True))[:1]

[(8207, {'tipo_no': 'sem_descricao'})]

In [9]:
print('número de vértices: ',D.number_of_nodes())
print('número de arestas: ',D.number_of_edges())

número de vértices:  9221
número de arestas:  6424


In [10]:
#buscando os atributos do vértice ocorrência
query = """SELECT 
                O.NRO_INT_OCORRENCIA,
                F.TXT_DESCRICAO, 
                --K66.K66_NRO_INT,
                O.DATA_FATO,
                O.HORA_FATO,
                K110.K110_BAIRRO_GEO,
                K110.K110_LAT, 
                K110.K110_LNG
          FROM 
                GESEG_TARGET.GESEG_VER_OCORRENCIA O
               LEFT JOIN GESEG_STG.KICC66D K66 ON O.NRO_INT_OCORRENCIA = K66.K66_NRO_INT_OCOR
               LEFT JOIN GESEG_STG.KICC110D K110 ON K66.K66_NRO_INT = K110.K110_NRO_INT AND K66.K66_ORG_REG = K110.K110_ORG_REG AND K66.K66_AA_REGISTRO = K110.K110_AA_REGISTRO
               LEFT JOIN GESEG_TARGET.GESEG_DIM_GRUPO_FATO F ON O.NRO_INT_FATO = F.NRO_INT_FATO
         WHERE 
                IND_TENTATIVA = 'Consumado'	
                AND YEAR(DATA_FATO) = 2021
                AND MONTH(DATA_FATO) = 1
                AND DAY(DATA_FATO) = 18;"""
                #AND HORA_REGISTRO > '20:00'

In [11]:
#TRANSFERINDO DADOS PARA TABELA
df_node = pd.read_sql_query(query, conn)

In [12]:
#Criando shape e cor diferente para os nós ocorrência
df_node['tipo_no'] = 'ocorrencia'
df_node['shape'] = 'triangle'
df_node['color'] = 'orange'
df_node['label'] = ' '
df_node['title'] = df_node['NRO_INT_OCORRENCIA'].astype(str)

#formatando o atributo Data
df_node['DATA_FATO'] = pd.to_datetime(df_node['DATA_FATO'])
df_node['DATA_FATO'] = df_node['DATA_FATO'].dt.strftime('%Y-%m-%d')

In [13]:
#transformando a tabela em um dicionário para atribuir os dados dos vértices 'ocorrencia'
node_ocorr_atrib = df_node.set_index("NRO_INT_OCORRENCIA").T.to_dict()
# atributo 
nx.set_node_attributes(D, node_ocorr_atrib)

In [14]:
# atributos dos nós participante ocorrência
query = """SELECT 
                DISTINCT(PO.NRO_INT_PESSOA_INTERESSE),
                --PO.NRO_INT_PARTICIPANTE_OCORRENCIA, 
                --PO.NRO_INT_OCORRENCIA,
                PI.NOME,
                DS.DESC_SEXO,
                PI.DATA_NASCIMENTO,
                PI.DATA_OBITO,
                PI.NOME_MAE,
                PI.NOME_PAI,
                PI.TXT_RG,
                PI.TXT_ALCUNHA,
                PI.COD_INDIVIDUO


          FROM   
                GESEG_TARGET.GESEG_VER_PARTICIPANTE_OCORRENCIA PO
                LEFT JOIN GESEG_TARGET.GESEG_VER_OCORRENCIA O ON PO.NRO_INT_OCORRENCIA = O.NRO_INT_OCORRENCIA
                LEFT JOIN GESEG_TARGET.GESEG_VER_PESSOA_INTERESSE PI ON PO.NRO_INT_PESSOA_INTERESSE = PI.NRO_INT_PESSOA_INTERESSE
                LEFT JOIN GESEG_TARGET.GESEG_DIM_SEXO DS ON PI.NRO_INT_SEXO = DS.NRO_INT_SEXO
          WHERE 
                
                IND_TENTATIVA = 'Consumado'	
                AND YEAR(DATA_FATO) = 2021
                AND MONTH(DATA_FATO) = 1
                AND DAY(DATA_FATO) = 18;"""

In [15]:
#TRANSFERINDO DADOS PARA TABELA
df_node2 = pd.read_sql_query(query, conn)

In [17]:
df_node2.rename(str.lower, axis='columns', inplace = True)
df_node2.head()

,nro_int_pessoa_interesse,nome,desc_sexo,data_nascimento,data_obito,nome_mae,nome_pai,txt_rg,txt_alcunha,cod_individuo
0,10302344,ZADIRA DA SILVA AZEREDO,Feminino,1927-01-18,NaT,MARIA FRAGA,OLIMPIO LUCIANO,8000062417,None,5346.0
1,5107446,CARMEN BENITES RODRIGUES,Feminino,1928-02-23,NaT,HORTENCIA MEIRELLES BENITES,HENRIQUE BENITES,9000080631,None,7141.0
2,318357,ANA MARIA DE OLIVEIRA,Feminino,1938-08-06,NaT,FLORENTINA DE OLIVEIRA,INACIO DO AMARAL,7000082383,None,7314.0
3,10147166,JORGE CARVALHO,Masculino,1930-01-04,2016-07-19,MARIA CARVALHO,ARMANDO CARVALHO,8000137417,JORGE SILVINO CARVALHO,12720.0
4,485074,CESAR ACOSTA DE CASTRO,Masculino,1942-08-07,NaT,ALDA ACOSTA DE CASTRO,VITERBO DE CASTRO,1000137578,None,12736.0


In [18]:
#Retirando o label dos nós
df_node2['tipo_no'] = 'pessoa'
df_node2['label'] = ' '
df_node2['title'] = df_node2['nro_int_pessoa_interesse'].astype(str)

#formatando o atributo Data_Nascimento e Data_obito
df_node2['data_nascimento'] = pd.to_datetime(df_node2['data_nascimento'])
df_node2['data_nascimento'] = df_node2['data_nascimento'].dt.strftime('%Y-%m-%d')

df_node2['data_obito'] = pd.to_datetime(df_node2['data_obito'])
df_node2['data_obito'] = df_node2['data_obito'].dt.strftime('%Y-%m-%d')

In [19]:
#transformando a tabela em um dicionário para atribuir os dados dos vértices 'ocorrencia'
node_ocorr_atrib2 = df_node2.set_index("nro_int_pessoa_interesse").T.to_dict()

In [20]:
# definindo atributos
nx.set_node_attributes(D, node_ocorr_atrib2)

In [21]:
# testando se todos os nós estão com o 'tipo' definido
teste2 = (n for n in D if D.nodes[n]['tipo_no']== 'sem_descricao')
print(sorted(teste2))

[]


## Co-autores

In [22]:
# gerando um subgrafo da ocorrência 38104786
T= nx.generators.ego_graph(D, 38095084, radius=1, center = True, undirected = True)

# plotar o grafo
nt = Network(height='750px', width='100%', notebook = True, directed = False)
nt.from_nx(T)
nt.show("t.html")

In [23]:
# Verificando os edges do subgrafo
sorted(T.edges(data = True, keys = True))

[(38095084, 853756, 0, {'tipo_participacao': 'Vítima', 'weight': 1}),
 (38095084, 3712999, 0, {'tipo_participacao': 'Testemunha', 'weight': 1}),
 (38095084, 3942682, 0, {'tipo_participacao': 'Condutor', 'weight': 1}),
 (38095084, 6319553, 0, {'tipo_participacao': 'Vítima', 'weight': 1}),
 (38095084, 6482231, 0, {'tipo_participacao': 'Vítima', 'weight': 1}),
 (38095084, 9409161, 0, {'tipo_participacao': 'Testemunha', 'weight': 1}),
 (38095084, 10781003, 0, {'tipo_participacao': 'Vítima', 'weight': 1}),
 (38095084, 13032480, 0, {'tipo_participacao': 'Vítima', 'weight': 1}),
 (38095084, 13193475, 0, {'tipo_participacao': 'Vítima', 'weight': 1}),
 (38095084,
  13359904,
  0,
  {'tipo_participacao': 'Adolescente infrator', 'weight': 1}),
 (38095084, 14177707, 0, {'tipo_participacao': 'Vítima', 'weight': 1}),
 (38095084, 14356510, 0, {'tipo_participacao': 'Suspeito', 'weight': 1})]

In [86]:
# selecionando nós só com co-autores
q = set(n for u,v,k,d in T.edges(data=True, keys = True)
               if d['tipo_participacao']=='Suspeito' 
               or d['tipo_participacao']=='Condutor'
               or d['tipo_participacao']=='Adolescente infrator'
               for n in (u, v)
               if T.nodes[n]['tipo_no'] == 'pessoa')
print(q)

{13359904, 3942682, 14356510}


In [88]:
# gerando um grafo ligando todos os nós co-autores de uma ocorrência
G = nx.complete_graph(q, nx.MultiDiGraph())

In [89]:
# verificando edges
G.edges(data=True)

OutMultiEdgeDataView([(13359904, 3942682, {}), (13359904, 14356510, {}), (3942682, 13359904, {}), (3942682, 14356510, {}), (14356510, 13359904, {}), (14356510, 3942682, {})])

In [90]:
# plotar o grafo de co-autores
nt = Network(height='750px', width='100%', notebook = True, directed = False)
nt.from_nx(G)
nt.show("g.html")

In [91]:
# definindo a relação de co-autores
for u,v,k in G.edges:
    G.edges[u,v,k]['tipo_participacao'] = 'co-autores'

In [92]:
# Relação está correta
G.edges(data=True)

OutMultiEdgeDataView([(13359904, 3942682, {'weight': 1, 'tipo_participacao': 'co-autores'}), (13359904, 14356510, {'weight': 1, 'tipo_participacao': 'co-autores'}), (3942682, 13359904, {'weight': 1, 'tipo_participacao': 'co-autores'}), (3942682, 14356510, {'weight': 1, 'tipo_participacao': 'co-autores'}), (14356510, 13359904, {'weight': 1, 'tipo_participacao': 'co-autores'}), (14356510, 3942682, {'weight': 1, 'tipo_participacao': 'co-autores'})])

In [93]:
# Unindo o grafo original (ocorrência -> pessoa-interesse) com o de co-autores
U= nx.compose(T, G)

In [94]:
# plotar o grafo completo
nt = Network(height='750px', width='100%', notebook = True, directed = False)
nt.from_nx(U)
nt.show("u.html")

In [90]:
conn.close()